<a href="https://colab.research.google.com/github/barabonda/SK-AI-FLY/blob/main/7%EC%A3%BC%EC%B0%A8/LangChain_05_Text_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai langchain tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 299.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninst


좌표계를 만들어주는 것이 embedding(놓) Model

In [2]:
import os

os.environ['OPENAI_API_KEY'] = '3a9d04966a3c43ca92de7be4b803d6f7'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://skyfly-001.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [13]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings_model = AzureOpenAIEmbeddings(
                        model='dev-text-embedding-ada-002'
)

In [14]:
embeddings = embeddings_model.embed_documents(
                {
                    '안녕하세요',
                    '제 이름은 홍길동 입니다.',
                    '이름이 무엇인가요?',
                    '랭체인은 유용합니다.',
                    'Hello World'
                }
              )
len(embeddings), len(embeddings[4])
#임베딩을 거치면 변환해서 엔코딩된다 좌표찍기(임베딩),좌표계의 거리가 1536이다

(5, 1536)

In [17]:
embedded_query_q = embeddings_model.embed_query('이 대화에 언급된 이름은 무엇입니까?')
embedded_query_a = embeddings_model.embed_query('이 대화에 언급된 이름은 홍길동 입니다.')
print(len(embedded_query_q),len(embedded_query_a))

1536 1536


cosine 맨하튼

In [20]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sin(A,B):
  return dot(A,B)/(norm(A)*norm(B))



In [25]:
print(cos_sin(embedded_query_q, embedded_query_a)) # 높은편
print(cos_sin(embedded_query_q, embeddings[1]))
print(cos_sin(embedded_query_q, embeddings[2]))


0.9085512700834164
0.9216955277777744
0.7748425804148096


# Huggingface Embedding

In [26]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00


In [33]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [34]:
embeddings = hf.embed_documents(
            [
                'today is monday',
                'weather is nice today',
                "what's the problem.",
                'langchain is useful',
                'Hello World',
                'my name is morris'
            ]
        )

In [43]:
BGE_qury_q = hf.embed_query('Hello? who is this?')
BGE_qury_a = hf.embed_query('hi this ;is harrison')

print(cos_sin(BGE_qury_q,BGE_qury_a))
print(cos_sin(BGE_qury_q,embeddings[1]))
print(cos_sin(BGE_qury_q,embeddings[2]))

0.8562599938924814
0.7469068749829496
0.770709978866335


In [ ]:
# 0.85 이정도면 꽤 높음